<a href="https://colab.research.google.com/github/romeroc223/SySistemas2025/blob/main/Parcial_2_SyS_2025_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install streamlit control matplotlib -q
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -O cloudflared
!chmod +x cloudflared
!mv cloudflared /usr/local/bin/cloudflared
!pip install yt-dlp -q
!apt-get install ffmpeg -y
!mkdir -p pages

--2025-12-03 05:33:35--  https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/cloudflare/cloudflared/releases/download/2025.11.1/cloudflared-linux-amd64 [following]
--2025-12-03 05:33:35--  https://github.com/cloudflare/cloudflared/releases/download/2025.11.1/cloudflared-linux-amd64
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/106867604/955e9d1b-ac5e-4188-8867-e5f53958a8fe?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-12-03T06%3A21%3A10Z&rscd=attachment%3B+filename%3Dcloudflared-linux-amd64&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-12-03

In [ ]:
%%writefile 0_Inicio.py
import streamlit as st

st.set_page_config(
    page_title="Parcial 2 - Señales y Sistemas 2025-II",
    page_icon="📘",
    layout="centered"
)

st.markdown("<h1 style='text-align: center; color: #004488;'>📘 Parcial 2 - Señales y Sistemas 2025-II</h1>", unsafe_allow_html=True)
st.markdown("---")

st.markdown("""
Esta aplicación presenta la solución completa del **Parcial 2 (2025-II)**, implementada con paneles interactivos de Streamlit.

### 📌 Contenido

#### 1️⃣ Demodulación en Amplitud (DSB-SC)
- Modulación y demodulación síncrona.
- **Filtrado pasa bajas implementado con FFT (Transformada Rápida de Fourier)**.

#### 2️⃣ Sistema Masa-Resorte y Equivalente Eléctrico
- Simulación de respuestas al **Impulso, Escalón y Rampa**.
- Análisis de amortiguamiento.

---
### 👤 Estudiante
- **Curso:** Señales y Sistemas
""")

st.sidebar.success("Selecciona un punto en el menú lateral.")

Overwriting 0_Inicio.py


In [ ]:
%%writefile pages/1_Demodulacion.py
import streamlit as st
import numpy as np
import matplotlib.pyplot as plt
from scipy.fft import fft, ifft, fftfreq, fftshift
from scipy.io import wavfile
import os
import subprocess
import yt_dlp

st.set_page_config(page_title="Punto 1: Demodulación DSB-SC", layout="wide")

st.markdown("<h1 style='text-align: center;'>📻 Punto 1: Demodulación de Amplitud (DSB-SC)</h1>", unsafe_allow_html=True)
st.markdown("---")

# --- FUNCIONES DE SOPORTE ---
def load_audio_from_youtube(url, duration_s=5, start_s=0):
    temp_wav = 'temp_audio.wav'
    cropped_wav = 'cropped.wav'
    try:
        ydl_opts = {'format': 'bestaudio/best', 'outtmpl': 'temp_audio', 'quiet': True,
                    'postprocessors': [{'key': 'FFmpegExtractAudio','preferredcodec': 'wav'}]}
        if os.path.exists(temp_wav): os.remove(temp_wav)

        # El archivo que descarga youtube-dl es temp_audio.wav
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([url])

        # Corregir el nombre del archivo si es necesario, o asegurar que se llame temp_audio.wav
        # Si yt-dlp guarda como temp_audio.mp3, esto fallará. Ajustamos para ser robustos.
        # Buscamos el archivo generado que termine en .wav
        downloaded_file = next((f for f in os.listdir('.') if f.startswith('temp_audio') and f.endswith('.wav')), 'temp_audio.wav')


        subprocess.run([
            'ffmpeg', '-i', downloaded_file, '-ss', str(start_s), '-t', str(duration_s),
            '-acodec', 'pcm_s16le', '-ar', '44100', '-ac', '1', cropped_wav, '-y'
        ], check=True, capture_output=True)

        fs, data = wavfile.read(cropped_wav)
        if data.ndim > 1: data = data.mean(axis=1)
        data = data / np.max(np.abs(data)) # Normalizar
        return data, fs
    except Exception as e:
        st.error(f"❌ Error descargando audio. Asegúrate de que la URL es correcta y ffmpeg está disponible: {e}")
        return None, None

def plot_spectrum(sig, fs, title, ax):
    N = len(sig)
    Y = fft(sig)
    freqs = fftfreq(N, 1/fs)
    Y_shift = fftshift(Y)
    freq_shift = fftshift(freqs)
    ax.plot(freq_shift, np.abs(Y_shift))
    ax.set_title(title)
    ax.set_xlabel("Frecuencia (Hz)")
    ax.set_xlim(-20000, 20000)
    ax.grid(True, alpha=0.3)

def fft_lowpass_filter(signal, fs, cutoff):
    """Implementación del filtro pasa bajas usando FFT (Requerimiento del parcial)"""
    N = len(signal)
    F = fft(signal)
    freqs = fftfreq(N, 1/fs)

    # Máscara ideal: pasa si |f| < cutoff
    mask = np.abs(freqs) < cutoff
    F_filtered = F * mask

    return ifft(F_filtered).real

# --- INTERFAZ Y PROCESAMIENTO ---
st.sidebar.header("Parámetros de Demodulación")
# Se utiliza una URL de ejemplo genérica, el usuario debe cambiarla
url = st.sidebar.text_input("URL YouTube", "https://www.youtube.com/watch?v=dQw4w9WgXcQ")
start_time = st.sidebar.slider("Inicio (s)", 0, 60, 10)
fc = st.sidebar.slider("Frecuencia Portadora (fc)", 3000, 15000, 10000)
cutoff = st.sidebar.slider("Corte Filtro LPF (Hz)", 1000, 8000, 4000)

if st.sidebar.button("Simular Demodulación"):
    with st.spinner("Procesando audio y simulando..."):
        m_t, fs = load_audio_from_youtube(url, duration_s=5, start_s=start_time)

        if m_t is not None:
            t = np.linspace(0, len(m_t)/fs, len(m_t), endpoint=False)

            # Etapa 1: Modulación DSB-SC
            carrier = np.cos(2 * np.pi * fc * t)
            x_t = m_t * carrier

            # Etapa 2: Mezcla Sincrónica (Demodulación)
            y_t = x_t * carrier

            # Etapa 3: Filtrado (Vía FFT)
            m_recuperada = fft_lowpass_filter(y_t, fs, cutoff)

            # Etapa 4: Escalamiento (x2 para recuperar amplitud Ac=1)
            m_final = m_recuperada * 2

            # --- VISUALIZACIÓN ---
            st.subheader("1. Señal de Mensaje Original m(t)")
            colA, colB = st.columns(2)
            with colA: figA, axA = plt.subplots(figsize=(6, 3)); axA.plot(t[:1000], m_t[:1000]); axA.set_title("Tiempo (Zoom)"); st.pyplot(figA)
            with colB: figB, axB = plt.subplots(figsize=(6, 3)); plot_spectrum(m_t, fs, "Espectro Original", axB); st.pyplot(figB)
            st.audio(m_t, sample_rate=fs)

            st.subheader("2. Señal Modulada x(t)")
            colC, colD = st.columns(2)
            with colC: figC, axC = plt.subplots(figsize=(6, 3)); axC.plot(t[:200], x_t[:200], color='orange'); axC.set_title("Tiempo (Zoom)"); st.pyplot(figC)
            # ----------------------------------------------------------------------------------
            # CORRECCIÓN DE TYPO: Se asegura que se use axD para la columna D (Espectro Modulado)
            # ----------------------------------------------------------------------------------
            with colD: figD, axD = plt.subplots(figsize=(6, 3)); plot_spectrum(x_t, fs, "Espectro Modulado", axD); st.pyplot(figD)

            st.subheader("3. Salida del Mezclador y(t) (Antes del Filtro)")
            colE, colF = st.columns(2)
            with colE: figE, axE = plt.subplots(figsize=(6, 3)); axE.plot(t[:200], y_t[:200], color='green'); axE.set_title("Tiempo (Zoom)"); st.pyplot(figE)
            with colF: figF, axF = plt.subplots(figsize=(6, 3)); plot_spectrum(y_t, fs, "Espectro (Banda base + 2fc)", axF); st.pyplot(figF)

            st.subheader(f"4. Señal Recuperada m'(t) (Filtro FFT corte {cutoff}Hz)")
            colG, colH = st.columns(2)
            with colG: figG, axG = plt.subplots(figsize=(6, 3)); axG.plot(t[:1000], m_final[:1000], color='purple'); axG.set_title("Tiempo (Zoom)"); st.pyplot(figG)
            with colH: figH, axH = plt.subplots(figsize=(6, 3)); plot_spectrum(m_final, fs, "Espectro Recuperado", axH); st.pyplot(figH)
            st.audio(m_final, sample_rate=fs)

Overwriting pages/1_Demodulacion.py


In [ ]:
%%writefile pages/2_Sistema.py
import streamlit as st
import numpy as np
import scipy.signal as signal
import matplotlib.pyplot as plt
from control import matlab

st.set_page_config(page_title="Punto 2: Sistema Dinámico", layout="wide")

st.markdown("<h1 style='text-align: center;'>⚙️ Punto 2: Sistema Mecánico y Equivalente Eléctrico</h1>", unsafe_allow_html=True)
st.markdown("---")

# --- MODELADO TEÓRICO ---
st.markdown("### 📝 Función de Transferencia (Lazo Abierto $G(s)$)")
st.markdown("El sistema masa-resorte-amortiguador tiene la ecuación diferencial: $m\\ddot{y} + c\\dot{y} + ky = F_E$")
st.latex(r"G(s) = H_{mec}(s) = \frac{Y(s)}{F(s)} = \frac{1/m}{s^2 + \frac{c}{m}s + \frac{k}{m}}")

# Muestra el sistema mecánico y el equivalente.
st.image("https://i.imgur.com/8Qj8n60.png", caption="Sistema Masa-Resorte-Amortiguador y Circuito Equivalente RLC.")

# --- CONTROLES ---
st.sidebar.header("Parámetros del Sistema (m, k, c)")
m = st.sidebar.number_input("Masa (m) [kg]", value=1.0, step=0.1)
k = st.sidebar.number_input("Resorte (k) [N/m]", value=10.0, step=1.0)

modo = st.sidebar.radio("Modo de Amortiguamiento", ["Subamortiguado", "Crítico", "Sobreamortiguado"])

wn = np.sqrt(k/m)
cc = 2 * np.sqrt(k*m)

if modo == "Subamortiguado":
    c = st.sidebar.slider("Coeficiente c", 0.01*cc, cc*0.99, cc*0.3)
elif modo == "Crítico":
    c = cc
    st.sidebar.info(f"c fijado a {c:.2f} para amortiguamiento crítico")
elif modo == "Sobreamortiguado":
    c = st.sidebar.slider("Coeficiente c", cc*1.01, cc*5.0, cc*1.5)

zeta = c / (2 * np.sqrt(k*m))

# --- FUNCIÓN DE TRANSFERENCIA ---
# 1. Lazo Abierto (Open-Loop - OL)
num_ol = [1/m]
den_ol = [1, c/m, k/m]
sys_ol = signal.TransferFunction(num_ol, den_ol)

# 2. Lazo Cerrado (Closed-Loop - CL): Asumimos retroalimentación unitaria H=1: G_CL = G / (1 + G)
# G(s) = (1/m) / (s^2 + (c/m)s + k/m)
# G_CL(s) = (1/m) / (s^2 + (c/m)s + k/m + 1/m)
num_lc = num_ol
den_lc = [1, c/m, (k + 1) / m]
sys_lc = signal.TransferFunction(num_lc, den_lc)

# --- CÁLCULO DE EQUIVALENCIA ELÉCTRICA (No se modifica) ---
C_elec = 1.0
L_elec = (m/k) / C_elec
R_elec = (m/c) / C_elec if c != 0 else np.inf

st.markdown("### 📐 Parámetros del Sistema")
col1, col2, col3 = st.columns(3)
with col1: st.metric("Frecuencia Natural $\\omega_n$", f"{wn:.2f} rad/s")
with col2: st.metric("Factor de Amortiguamiento $\\zeta$", f"{zeta:.2f}")

st.markdown("#### **Equivalentes Eléctricos (asumiendo C=1F):**")
st.code(f"R = {R_elec:.3f} Ω | L = {L_elec:.3f} H | C = {C_elec:.1f} F")

# Tiempo
t = np.linspace(0, 15, 1000)

# Gráficas
st.markdown("---")
st.markdown("### 📊 Análisis del Comportamiento")
tab1, tab2, tab3 = st.tabs(["⚡ Respuestas Temporales", "🌀 Polos y Ceros", "📈 Diagrama de Bode"])

with tab1:
    st.subheader("Respuestas al Impulso, Escalón y Rampa (Lazo Abierto vs. Cerrado)")

    fig1, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(18, 5))

    # 1. Impulso
    t_imp, y_imp_ol = signal.impulse(sys_ol, T=t)
    _, y_imp_lc = signal.impulse(sys_lc, T=t)
    ax1.plot(t_imp, y_imp_ol, 'r', label="Lazo Abierto (OL)"); ax1.plot(t_imp, y_imp_lc, 'r--', label="Lazo Cerrado (CL)")
    ax1.set_title("Respuesta al Impulso"); ax1.set_xlabel("Tiempo (s)"); ax1.grid(True); ax1.legend()

    # 2. Escalón
    t_step, y_step_ol = signal.step(sys_ol, T=t)
    _, y_step_lc = signal.step(sys_lc, T=t)
    ax2.plot(t_step, y_step_ol, 'b', label="Lazo Abierto (OL)"); ax2.plot(t_step, y_step_lc, 'b--', label="Lazo Cerrado (CL)")
    ax2.set_title("Respuesta al Escalón"); ax2.set_xlabel("Tiempo (s)"); ax2.grid(True); ax2.legend()

    # 3. Rampa: G(s)/s
    sys_ramp_ol = signal.TransferFunction(num_ol, np.polymul(den_ol, [1, 0]))
    t_ramp, y_ramp_ol = signal.step(sys_ramp_ol, T=t)

    sys_ramp_lc = signal.TransferFunction(num_lc, np.polymul(den_lc, [1, 0]))
    _, y_ramp_lc = signal.step(sys_ramp_lc, T=t)

    ax3.plot(t_ramp, y_ramp_ol, 'g', label="Lazo Abierto (OL)")
    ax3.plot(t_ramp, y_ramp_lc, 'g--', label="Lazo Cerrado (CL)")
    ax3.plot(t, t, 'k:', label="Rampa Entrada")
    ax3.set_title("Respuesta a la Rampa"); ax3.set_xlabel("Tiempo (s)"); ax3.grid(True); ax3.legend()

    st.pyplot(fig1)

with tab2:
    st.subheader("Mapa de Polos y Ceros")
    fig_pz, ax_pz = plt.subplots()

    # Polos Lazo Abierto
    ax_pz.scatter(np.real(sys_ol.poles), np.imag(sys_ol.poles), marker='x', s=100, color='blue', label='Polos OL')
    # Polos Lazo Cerrado
    ax_pz.scatter(np.real(sys_lc.poles), np.imag(sys_lc.poles), marker='o', s=100, color='red', facecolors='none', edgecolors='red', label='Polos CL')

    ax_pz.axhline(0, color='black', lw=1); ax_pz.axvline(0, color='black', lw=1)
    ax_pz.set_title("Mapa de Polos y Ceros"); ax_pz.set_xlabel("Eje Real"); ax_pz.set_ylabel("Eje Imaginario")
    ax_pz.grid(True); ax_pz.legend()
    st.pyplot(fig_pz)

with tab3:
    st.subheader("Diagrama de Bode (Lazo Abierto vs. Cerrado)")
    w_ol, mag_ol, phase_ol = signal.bode(sys_ol)
    w_lc, mag_lc, phase_lc = signal.bode(sys_lc)

    fig_bode, (ax_m, ax_p) = plt.subplots(2, 1, figsize=(8, 6), sharex=True)

    # Magnitud
    ax_m.semilogx(w_ol, mag_ol, label="Lazo Abierto"); ax_m.semilogx(w_lc, mag_lc, '--', label="Lazo Cerrado")
    ax_m.set_ylabel("Magnitud (dB)"); ax_m.grid(True); ax_m.legend()

    # Fase
    ax_p.semilogx(w_ol, phase_ol, label="Lazo Abierto"); ax_p.semilogx(w_lc, phase_lc, '--', label="Lazo Cerrado")
    ax_p.set_ylabel("Fase (deg)"); ax_p.set_xlabel("Frecuencia (rad/s)"); ax_p.grid(True)
    st.pyplot(fig_bode)

Overwriting pages/2_Sistema.py


In [ ]:
# =========================================================
#
# =========================================================

# PASO 1: Asegurar que los procesos estén limpios
print("Deteniendo y limpiando procesos anteriores...")
!pkill -f streamlit || true
!pkill -f cloudflared || true

# PASO 2: Lanzar Streamlit y Cloudflared Tunnel
print("Lanzando Streamlit desde 0_Inicio.py...")
!streamlit run 0_Inicio.py &>/content/logs.txt &
!cloudflared tunnel --url http://localhost:8501 > /content/cloudflared.log 2>&1 &

# PASO 3: Mostrar URL
import time, re
time.sleep(12)

try:
    with open('/content/cloudflared.log') as f:
        log_data = f.read()
        match = re.search(r"https://[-\w.]+\.trycloudflare\.com", log_data)
        if match:
            print("✅ Tu app está disponible en:", match.group(0))
        else:
            print("❌ No se pudo encontrar el enlace. Intenta ejecutar de nuevo.")
except FileNotFoundError:
    print("❌ Archivo de log no encontrado.")

Deteniendo y limpiando procesos anteriores...
^C
^C
Lanzando Streamlit desde 0_Inicio.py...
✅ Tu app está disponible en: https://has-mattress-change-messenger.trycloudflare.com
